In [3]:
"""
If necessary, uncomment and install:
"""
!pip install wbdata
!pip install cufflinks


import wbdata
import cufflinks as cf
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt

In [16]:
SOURCE = 40
indicators = wbdata.get_indicator(source=SOURCE)
indicators

id                 name
-----------------  -------------------------------------------------------------------
SH.DTH.0509        Number of deaths ages 5-9 years
SH.DTH.1014        Number of deaths ages 10-14 years
SH.DTH.1519        Number of deaths ages 15-19 years
SH.DTH.2024        Number of deaths ages 20-24 years
SH.DTH.IMRT        Number of infant deaths
SH.DTH.IMRT.FE     Number of infant deaths, female
SH.DTH.IMRT.MA     Number of infant deaths, male
SH.DTH.MORT        Number of under-five deaths
SH.DTH.MORT.FE     Number of under-five deaths, female
SH.DTH.MORT.MA     Number of under-five deaths, male
SH.DTH.NMRT        Number of neonatal deaths
SH.DYN.0509        Probability of dying among children ages 5-9 years (per 1,000)
SH.DYN.1014        Probability of dying among adolescents ages 10-14 years (per 1,000)
SH.DYN.1519        Probability of dying among adolescents ages 15-19 years (per 1,000)
SH.DYN.2024        Probability of dying among youth ages 20-24 years (per 1,000)

In [ ]:
SOURCE = 87 
# 40 - "Population estimates and projections"
# 87 - 'Country Climate and Development Report'

indicators = wbdata.get_indicator(source=SOURCE)
indicators

In [11]:
# Give variable for clarity
variable_labels = {'SP.URB.GROW' : 'upg', 'SP.RUR.TOTL.ZG': 'rpg'}

# world = wbdata.get_dataframe(variable_labels, country="WLD")
prk_pop_grow = wbdata.get_dataframe(variable_labels, country='PRK')
prk_pop_grow
# Date index is of type string; change to integers
# nzl_gdp.index = world.index.astype(int)
# usa_gdp.index = world.index.astype(int)


,upg,rpg
date,,
2020,0.834712,-0.216466
2019,0.834074,-0.163550
2018,0.827629,-0.108409
2017,0.817680,-0.057201
2016,0.806606,-0.011742
...,...,...
1964,4.043233,0.033564
1963,3.917962,-0.089348
1962,4.090284,0.082355


In [13]:
fig = px.line(prk_pop_grow, x=prk_pop_grow.index, y=['upg', 'rpg'])
fig['layout']['xaxis']['autorange'] = "reversed"
fig.show()

Growth rates are almost the same between 1971-1979. What might be the cause?
Mirroring each other in the opposite directions between 1980 and 2000.
Why do they both fall between 2016 and 2020?
However, this visualization might be misleading because as we see below, the urban population remains relatively steady at around 50 M. 

In [14]:
# Give variable for clarity
variable_labels = {'SP.URB.TOTL' : 'urban pop', 'SP.RUR.TOTL': 'rural pop'}

world_pop_urban_rural = wbdata.get_dataframe(variable_labels, country="WLD")
prk_pop_urban_rural = wbdata.get_dataframe(variable_labels, country='PRK')
# Date index is of type string; change to integers
# nzl_gdp.index = world.index.astype(int)
# usa_gdp.index = world.index.astype(int)

In [15]:
fig = px.line(prk_pop_urban_rural, x=prk_pop_urban_rural.index, y=['urban pop', 'rural pop'])
fig['layout']['xaxis']['autorange'] = "reversed"
fig.show()